In [11]:
import pygame
import sys

pygame.init()

# 화면 크기 설정
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("라면 게임")

# 색상 및 폰트 설정
YELLOW = (255, 255, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)

font_path = "C:/Windows/Fonts/malgun.ttf"
font = pygame.font.Font(font_path, 40)
small_font = pygame.font.Font(font_path, 30)

# 이미지 설정
background_img = pygame.transform.scale(pygame.image.load("img.png"), (width, height))
ending_img = pygame.transform.scale(pygame.image.load("ending.png"), (width, height))
instructions_img = pygame.transform.scale(pygame.image.load("First.png"), (width, height))

# 재료 이미지
kettle_img = pygame.transform.scale(pygame.image.load("kettle.png"), (200, 200))
pot_img = pygame.transform.scale(pygame.image.load("pot.png"), (200, 200))
egg_img = pygame.transform.scale(pygame.image.load("egg.png"), (100, 100))
ramen_img = pygame.transform.scale(pygame.image.load("ramen.png"), (100, 100))
pa_img = pygame.transform.scale(pygame.image.load("pa.png"), (100, 100))
supoo_img = pygame.transform.scale(pygame.image.load("supoo.png"), (100, 100))
supoo_1_img = pygame.transform.scale(pygame.image.load("supoo1.png"), (100, 100))

score = 0


class Button:
    def __init__(self, x, y, width, height, text, action=None, color=(255, 232, 73)):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.action = action
        self.color = color

    def draw(self, screen):
        pygame.draw.rect(screen, self.color, self.rect)
        text_surface = font.render(self.text, True, BLACK)
        screen.blit(text_surface, (self.rect.centerx - text_surface.get_width() // 2,
                                   self.rect.centery - text_surface.get_height() // 2))

    def is_clicked(self, pos):
        return self.rect.collidepoint(pos)


class Block:
    def __init__(self, color=None, x=0, y=0, size=50, image=None):
        self.color = color
        self.rect = pygame.Rect(x, y, size, size)
        self.dragging = False
        self.image = image
        if image:
            self.rect.size = image.get_size()

    def draw(self, screen):
        if self.image:
            screen.blit(self.image, self.rect.topleft)
        else:
            pygame.draw.rect(screen, self.color, self.rect)

    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN and self.rect.collidepoint(event.pos):
            self.dragging = True
            mouse_x, mouse_y = event.pos
            self.offset_x = self.rect.x - mouse_x
            self.offset_y = self.rect.y - mouse_y
        elif event.type == pygame.MOUSEBUTTONUP:
            self.dragging = False
        elif event.type == pygame.MOUSEMOTION and self.dragging:
            mouse_x, mouse_y = event.pos
            self.rect.x = mouse_x + self.offset_x
            self.rect.y = mouse_y + self.offset_y


def show_score():
    score_text = font.render(f"점수: {score}원", True, BLACK)
    screen.blit(score_text, (20, 20))


def evaluate_ramen(cooking_time, correct_order):
    global score
    if not correct_order:
        score += 2000
    elif cooking_time >= 10:
        score += 0
    elif cooking_time >= 4:
        score += 3000
    elif cooking_time >= 2:
        score += 1000
    return True


def ending_screen():
    restart_button = Button(550, 500, 200, 50, "다시하기")

    running = True
    while running:
        screen.blit(ending_img, (0, 0))
        restart_button.draw(screen)
        pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN and restart_button.is_clicked(event.pos):
                loading_screen()
                return


def start_game():
    global score
    score = 0

    yellow_block = Block(image=egg_img, x=100, y=100)
    blue_block = Block(image=ramen_img, x=300, y=100)
    green_block = Block(image=pa_img, x=500, y=100)
    soup_block = Block(image=supoo_img, x=700, y=100)
    kettle = Block(image=kettle_img, x=500, y=height - 350)

    pot_positions = [(100, 250), (300, 250), (100, 400), (300, 400)]
    pot_list = [Block(image=pot_img, x=x, y=y) for x, y in pot_positions]
    submit_button = Button(650, 500, 120, 50, "라면 제출")

    start_ticks = pygame.time.get_ticks()
    total_time = 60
    cooking_time = 0
    correct_order = True
    game_over = False

    while not game_over:
        elapsed_time = (pygame.time.get_ticks() - start_ticks) / 1000
        remaining_time = max(0, total_time - elapsed_time)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if submit_button.is_clicked(event.pos):
                    evaluate_ramen(cooking_time, correct_order)
            yellow_block.handle_event(event)
            blue_block.handle_event(event)
            green_block.handle_event(event)
            soup_block.handle_event(event)
            kettle.handle_event(event)
            for pot in pot_list:
                pot.handle_event(event)

        screen.fill(WHITE)

        # 냄비 먼저 그림 (뒤에)
        for pot in pot_list:
            pot.draw(screen)

        # 재료들과 주전자 그림 (앞에)
        yellow_block.draw(screen)
        blue_block.draw(screen)
        green_block.draw(screen)
        soup_block.draw(screen)
        kettle.draw(screen)

        submit_button.draw(screen)
        show_score()

        # 남은 시간 바
        bar_width = 300
        bar_height = 30
        time_ratio = remaining_time / total_time
        pygame.draw.rect(screen, GRAY, (250, 20, bar_width, bar_height))
        pygame.draw.rect(screen, (233, 55, 113), (250, 20, int(bar_width * time_ratio), bar_height))

        if remaining_time <= 0:
            game_over = True

        pygame.display.update()

    pygame.time.wait(3000)
    ending_screen()


def loading_screen():
    start_button = Button(300, 250, 200, 50, "Start", start_game)
    instructions_button = Button(300, 350, 200, 50, "게임 설명")
    game_button = Button(580, 500, 200, 50, "게임으로", start_game)

    show_instructions_flag = False
    running = True

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.MOUSEBUTTONDOWN:
                pos = event.pos
                if not show_instructions_flag:
                    if start_button.is_clicked(pos):
                        start_game()
                        return
                    elif instructions_button.is_clicked(pos):
                        show_instructions_flag = True
                else:
                    if game_button.is_clicked(pos):
                        start_game()
                        return
                    else:
                        show_instructions_flag = False

        screen.blit(background_img, (0, 0))

        if show_instructions_flag:
            screen.blit(instructions_img, (0, 0))
            game_button.draw(screen)
        else:
            start_button.draw(screen)
            instructions_button.draw(screen)

        pygame.display.update()

    pygame.quit()
    sys.exit()


loading_screen()


SystemExit: 